# Tâche 4

In [82]:
import numpy as np
from mp_api.client import MPRester
from pymatgen.core.operations import SymmOp
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from jupyter_jsmol.pymatgen import quick_view
from lmapr1492 import plot_brillouin_zone, get_plot_bs, get_plot_dos, get_plot_bs_and_dos, get_branch_wavevectors
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from IPython.display import Image

In [83]:
mp_key = "0xG7dCsc4AADF0PgqeiDeWIJa0gtg1Pd"
mp_id = "mp-9166"

In [84]:
with MPRester(mp_key) as m:
    struct = m.get_structure_by_material_id(mp_id)
prim_struct = SpacegroupAnalyzer(struct).get_primitive_standard_structure()

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

# 1) Réseau direct

Afin d'obtenir les vecteurs de base du réseau, nous avons utilisé la maille primitive. La cellule unité ou maille primitive est défini comme étant la plus petit partie du réseau permettant de recomposer ce-dernier entièrement. Les vecteurs de base du réseau sont donc les vecteurs fondamentaux qui définissent la répétition périodique de la structure. Ils permettent de générer l’ensemble des nœuds du réseau par translation.


In [95]:
R_direct = prim_struct.lattice
## vecteurs de base : 
A,B,C= np.array(R_direct.matrix)
print("Vecteurs de base du réseau direct:\n")
print(f"a = {A}")
print(f"b = {B}")
print(f"c = {C}\n")
## longueurs des vecteurs de base:
a,b,c = R_direct.abc
## paramètres de maille : 
print("Paramètres de maille du réseau direct:\n ")
print(f"||a|| = {round(a,4)} Å")
print(f"||b|| = {round(b,4)} Å")
print(f"||c|| = {round(c,4)} Å")
print(f"α = {round(R_direct.alpha,3)}°")
print(f"β = {round(R_direct.beta,3)}°")
print(f"γ = {round(R_direct.gamma,3)}°")








Vecteurs de base du réseau direct:

a = [ 4.80866755 -1.63004458  0.        ]
b = [4.80866755 1.63004458 0.        ]
c = [4.25611421 0.         2.76872187]

Paramètres de maille du réseau direct:
 
||a|| = 5.0774 Å
||b|| = 5.0774 Å
||c|| = 5.0774 Å
α = 37.451°
β = 37.451°
γ = 37.451°


# 2) Réseau réciproque

Le réseau réciproque est défini de manière à représenter les propriétés de diffraction des cristaux. Il s'agit de l'ensemble de tous les vecteurs K qui vérifient la relation suivante : $$e^{i \mathbf{K} \cdot \mathbf{R}} = 1  $$ 
où R est le vecteur position du réseau.

Les paramètres de maille et les vecteurs de bases du réseau peuvent être obtenus à l'aide des formules ci-dessous.




In [87]:

volume = np.dot(A,np.cross(B,C))
a_prime = (2*np.pi/volume)*np.cross(B,C)
b_prime = (2*np.pi/volume)* np.cross(C,A)
c_prime = (2*np.pi/volume)* np.cross(A,B)
 

print("Vecteurs de base du réseau réciproque:\n")
print(f"a' = {a_prime}")
print(f"b' = {b_prime}")
print(f"c' = {c_prime}\n")
## paramètres de maille : 
print("Paramètres de maille du réseau réciproque:\n ")
print(f"||a'|| = {round(np.sqrt(np.dot(a_prime,a_prime)),4)} Å⁻¹")
print(f"||b'|| = {round(np.sqrt(np.dot(b_prime,b_prime)),4)} Å⁻¹")
print(f"||c'|| = {round(np.sqrt(np.dot(c_prime,c_prime)),4)} Å⁻¹")

## angles obtenus d'une autre manière: 

R_reci = prim_struct.lattice.reciprocal_lattice
print(f"α = {round(R_reci.alpha,3)}°")
print(f"β = {round(R_reci.beta,3)}°")
print(f"γ = {round(R_reci.gamma,3)}°")



Vecteurs de base du réseau réciproque:

a' = [ 0.65331875 -1.92730475 -1.00428982]
b' = [ 0.65331875  1.92730475 -1.00428982]
c' = [-0.          0.          2.26934506]

Paramètres de maille du réseau réciproque:
 
||a'|| = 2.2693 Å⁻¹
||b'|| = 2.2693 Å⁻¹
||c'|| = 2.2693 Å⁻¹
α = 116.266°
β = 116.266°
γ = 116.266°


# 3) Sytème cristallin

In [93]:
s = SpacegroupAnalyzer(struct).get_crystal_system() 
print("Système cristallin :", s)



Système cristallin : trigonal


# 4) Type de maille


In [90]:
t = SpacegroupAnalyzer(struct).get_lattice_type()
print("Type de maille :", t)


Type de maille : rhombohedral


# 5) groupe ponctuel

En cristallographie, l'ensemble des opérations ponctuelles de symétie associées à un cristal forme au sens mathématique un groupe G. Celui-ci est muni de plusieurs propriétés : 
1) Le groupe est muni d’une loi de composition interne: ∀X,Y ∈ 𝒢 : Z = X Y ∈ 𝒢 
2) La loi de composition interne est associative: ∀X,Y,Z ∈ 𝒢 : (X Y ) Z = X (Y Z ) 
3) Il existe un élément neutre (l’identité E): ∀X ∈ 𝒢 : X E = E X = X 
4) Pour tout élément du groupe, il existe un élément inverse: ∀X ∈ 𝒢  ∃ X-1 ∈ 𝒢  tel que X X-1 = X-1 X = E

En 2D, il existe 10 groupes ponctuels différents et en 3D, il en existe 32 différents.


In [94]:

g = SpacegroupAnalyzer(struct).get_point_group_symbol()
print("Groupe ponctuel 3D :", g)

Groupe ponctuel 3D : -3m
